In [26]:
import random
from datetime import datetime
from textwrap import dedent
import os
from variables import posibles_variables

def generar_script_airflow(posibles_variables, lineas_offset):
    # Plantilla base del script
    script_base = dedent("""
    from airflow import DAG
    from airflow.operators.python_operator import PythonOperator
    from airflow.models import Variable
    from datetime import datetime

    # Definir DAG
    default_args = {
        'owner': 'airflow',
        'depends_on_past': False,
        'start_date': datetime(2023, 8, 9),
        'retries': 1,
    }

    dag = DAG(
        'conexion_varia_{index}',
        default_args=default_args,
        description='Un DAG que utiliza una variedad de variables de Airflow',
        schedule_interval=None,
    )

    # Función para realizar la tarea
    def realizar_tarea(**kwargs):
        # Capturar variables
    """)

    # Seleccionar un número aleatorio de variables de la lista de posibles variables
    numero_variables = random.randint(1, lineas_offset)
    variables_seleccionadas = random.sample(posibles_variables, numero_variables)

    # Generar script con las variables seleccionadas
    script_variables = "\n".join([f"    {var} = Variable.get('{var}', default_var='')" for var in variables_seleccionadas])
    script_tareas = dedent(f"""
    # Asignar variables a funciones
{script_variables}

    print("Variables cargadas exitosamente.")
    """)

    # Definir la tarea de Python
    script_tareas += dedent("""
        realizar_tarea_task = PythonOperator(
            task_id='realizar_tarea',
            provide_context=True,
            python_callable=realizar_tarea,
            dag=dag,
        )

        realizar_tarea_task
    """)

    # Generar el script completo
    script_completo = script_base + script_tareas.format(index=lineas_offset)
    return script_completo

def guardar_scripts(cantidad, posibles_variables, numero_variables, carpeta_salida):
    # Crear la carpeta de salida si no existe
    if not os.path.exists(carpeta_salida):
        os.makedirs(carpeta_salida)
    
    for i in range(cantidad):
        script = generar_script_airflow(posibles_variables, numero_variables)
        nombre_archivo = f"{carpeta_salida}/script_{i + 1}.py"
        
        with open(nombre_archivo, 'w') as archivo:
            archivo.write(script)
        print(f"Script guardado en: {nombre_archivo}")
        
cantidad_scripts = 13  # Número de scripts a generar
numero_variables = 10  # Número de variables por script
carpeta_salida = "scripts_airflow"

guardar_scripts(cantidad_scripts, posibles_variables, numero_variables, carpeta_salida)


Script guardado en: scripts_airflow/script_1.py
Script guardado en: scripts_airflow/script_2.py
Script guardado en: scripts_airflow/script_3.py
Script guardado en: scripts_airflow/script_4.py
Script guardado en: scripts_airflow/script_5.py
Script guardado en: scripts_airflow/script_6.py
Script guardado en: scripts_airflow/script_7.py
Script guardado en: scripts_airflow/script_8.py
Script guardado en: scripts_airflow/script_9.py
Script guardado en: scripts_airflow/script_10.py
Script guardado en: scripts_airflow/script_11.py
Script guardado en: scripts_airflow/script_12.py
Script guardado en: scripts_airflow/script_13.py


In [29]:
import os
import pandas as pd

def buscar_variable_get(directorio):
    datos = []

    # Recorrer todos los archivos en el directorio
    for nombre_archivo in os.listdir(directorio):
        if nombre_archivo.endswith('.py'):
            ruta_archivo = os.path.join(directorio, nombre_archivo)

            with open(ruta_archivo, 'r') as archivo:
                lineas = archivo.readlines()

                # Buscar las líneas que contienen 'Variable.get'
                for numero_linea, linea in enumerate(lineas, start=1):
                    if 'Variable.get' in linea:
                        datos.append({
                            'Nombre del Script': nombre_archivo,
                            'Número de Línea': numero_linea,
                            'Contenido de la Línea': linea.strip()
                        })

    # Crear un DataFrame de pandas
    df = pd.DataFrame(datos)
    return df

# Ejemplo de uso
directorio = '/notebooks/recursos/Integracion GitLab/scripts_airflow'  # Cambia esto al directorio que desees
df_resultado = buscar_variable_get(directorio)
print(df_resultado)


FileNotFoundError: [Errno 2] No such file or directory: '/notebooks/recursos/Integracion GitLab/scripts_airflow'